## Python Workshop - Dash Tutorials 

In [2]:
!pip install dash
!pip install dash_renderer
!pip install dash_html_components
!pip install dash_core_components

In [1]:
import pandas as pd
import requests 
import datetime
import dash 
from dash import html 
from dash.dependencies import Input, Output
import plotly.express as px 
from dash import dcc

### 1st Step: Create A simple Dash Application with HTML Code and Dash Core Compenents

DCC Examples include: (Dropdown, Checklist, DatePickerRange, Download, Rangeslider, Upload, etc. )
For Full list refer to documentations: https://dash.plotly.com/dash-core-components/clipboard


In [46]:
app = dash.Dash("myApp")  #Intiate you Application 


app.layout = html.Div([
    
    html.H1(children = 'Hello Dash!!',
            style= {'textAlign': 'center'}),
    html.Div('Dash Stuff'),
    
    dcc.Graph(
        id = 'samplechart',
        figure = {
            'data' : [{'x':[4,6,8], 'y':[12,16,18], "type":'bar', 'name':'HI'},
                     {'x':[4,6,8], 'y':[16,12,11], "type":'bar', 'name':'H2'}],
    
            'layout': {'font': 'Outfit',
                       'title' : 'xxx',
                       'plot_bgcolor':'#D3D3D3'}}),

    dcc.Checklist(
                    id='checklist',
        options = ['binance', 'coinbase']
                )
    ])



if __name__ == '__main__': 
    app.run_server()

### Step 2: Get to Know the dash Core components And Run Interactive Charts

In [3]:
# Typical CCData Volumes 

api_key = '' #Add API Key here
list_of_exchanges = ['binance', 'coinbase', 'kraken']

def exchanges_volume(list_of_exchanges, tsym='USD', limit=10):
    df_final = pd.DataFrame()
    for exchange in list_of_exchanges: 
        url = (f'https://min-api.cryptocompare.com/data/exchange/histoday'
               f'?tsym={tsym}&e={exchange}&limit={limit}&api_key={api_key}')
        df = pd.DataFrame(requests.get(url).json()['Data'])
        df.index = pd.to_datetime(df.time, unit='s')
        df_final = pd.concat([df_final, df[['volume']].rename(
            columns={'volume': exchange})], axis=1)
    return df_final 

exchanges_volume = exchanges_volume(list_of_exchanges)

df = exchanges_volume.stack().reset_index().rename(
        columns={'level_1': 'Exchange', 0: 'Value'})



In [49]:
app = dash.Dash("CCData Volumes")  

app.layout = html.Div([
    html.H1(children = 'CCData Volumes!!',
            style= {'textAlign': 'center'}),
    html.Div('Top Centralised Exchanges volumes'),
    
#     dcc.Graph(id = 'exchanges_volume', figure = {}),

#     dcc.Dropdown(
#                     id='dropdown',
#                     options = [
#                         {'label': i, 'value': i} for i in df.Exchange.unique()],
#                     value = [i for i in df.Exchange.unique()],
#                     multi =True,
#                     placeholder = 'Choose an Exchange'),
    
    html.Button("Download CSV", id="btn_csv"),
    dcc.Download(id="download-dataframe-csv"),

])


@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    prevent_initial_call=True,
)

def func(n_clicks):
    return dcc.send_data_frame(df.to_csv, "mydf.csv")


# # Decorater 
# @app.callback(
#     Output(component_id='exchanges_volume', component_property='figure'),
#     Input(component_id='dropdown', component_property='value'))

# # The function 
# def update_graph(selected_exchange):
#     dff = df.loc[df.Exchange.isin(selected_exchange)]
#     fig = px.bar(dff, x='time', y='Value', color='Exchange')

#     return fig

if __name__ == '__main__': 
    app.run_server()


In [4]:
app = dash.Dash("CCData Volumes") 

app.layout = html.Div([
    html.H1(children = 'CCData Volumes!!',
            style= {'textAlign': 'center'}),
    html.Div('Top Centralised Exchanges volumes'),
    
    dcc.Graph(id = 'exchanges_volume', figure = {}),

    dcc.DatePickerRange( 
    
                    id= 'dateslider')])

# Decorater 
@app.callback(
    Output(component_id='exchanges_volume', component_property='figure'),
    [Input(component_id='dateslider', component_property='start_date'),
    Input(component_id='dateslider', component_property='end_date')])

# The function 
def update_graph(start_date, end_date):
    if start_date == None: start_date = datetime.datetime(2023,8,2,0,0,0)
    if end_date == None: end_date = datetime.datetime(2023,8,10,0,0,0)
    dff = df.loc[(df.time> start_date) & (df.time < end_date)]
    fig = px.bar(dff, x='time', y='Value', color='Exchange')

    return fig

if __name__ == '__main__': 
    app.run_server()


### Step 3: Run Real Time Charts 
   

In [15]:

app = dash.Dash("CCData CCCAGG Price") 

app.layout = html.Div([
    html.H1(children = 'CCData CCCAGG Price!!',
            style= {'textAlign': 'center'}),    
    dcc.Graph(id = 'btc_price', figure = {}),

    dcc.Interval(id='interval-component',
                interval=1*1000, 
                    n_intervals=0)])
list_ = []
 
@app.callback(
    Output(component_id='btc_price', component_property='figure'),
    Input(component_id='interval-component', component_property='n_intervals'))

def update_graph(n):
    global list_
    url = 'https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD'
    price = requests.get(url).json()['USD']
    
    list_.append([datetime.datetime.now(), price])

    df = pd.DataFrame({'x_data':[i[0] for i in list_], 'y_data':[i[1] for i in list_]})
    fig = px.line(df, x='x_data', y='y_data', title="Bitcoin Price")
    
    return fig

if __name__ == '__main__': 
    app.run_server()
 

In [14]:
import requests
import pandas as pd
import datetime

def orderbook_data():
    url = f'https://min-api.cryptocompare.com/data/v2/ob/l2/snapshot?' \
          f'fsym=btc&tsym=usdt&e=binance&limit=2000&api_key={api_key}'

    response_data = requests.get(url).json()['Data']

    bids_data = pd.DataFrame(response_data['BID'])
    asks_data = pd.DataFrame(response_data['ASK'])

    bids_data['SIDE'] = 'BID'
    asks_data['SIDE'] = 'ASK'

    df = pd.concat([bids_data, asks_data])
    df = df.assign(TIME=datetime.datetime.now())

    return df


In [55]:
import plotly.graph_objs as go


app = dash.Dash("CCData OrderBook BTC-USDT on Binance") 

app.layout = html.Div([
    html.H1(children = 'CCData OrderBook Team is Cool!!',
            style= {'textAlign': 'center'}),

    
    dcc.Graph(id = 'orderbook_fig', figure = {}),

    dcc.Interval(id='interval-component',
                interval=1*1000, # in milliseconds
                    n_intervals=0)])

df_final = pd.DataFrame()

@app.callback(
    Output(component_id='orderbook_fig', component_property='figure'),
    Input(component_id='interval-component', component_property='n_intervals'))


def update_graph(n):
    global df_final 
    df = orderbook_data()
    df_final = pd.concat([df_final, df])
#     print(df_final)
    
    fig = go.Figure(data=go.Heatmap(
        x=df_final['TIME'], y=df_final['P'],
        z=df_final['Q'], colorscale='blackbody'))
    
    return fig

if __name__ == '__main__': 
    app.run_server()

[2023-08-18 15:55:31,759] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/hosammahmoud/anaconda3/lib/python3.10/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/Users/hosammahmoud/anaconda3/lib/python3.10/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/Users/hosammahmoud/anaconda3/lib/python3.10/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 60] Operation timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hosammahmoud/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 714, in urlopen
    httplib_response = self._make_request(
  File "/Users/hosammahmoud/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py", line 403, in _make_request
    self

In [4]:
import plotly.graph_objs as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

df = orderbook_data()

fig = go.Figure(data=go.Heatmap(
        x=df['TIME'], y=df['P'],
        z=df['Q'], colorscale='blackbody'))

fig.show()

In [26]:
help(html.Div)

Help on class Div in module dash.html.Div:

class Div(dash.development.base_component.Component)
 |  Div(children=None, id=undefined, n_clicks=undefined, n_clicks_timestamp=undefined, disable_n_clicks=undefined, key=undefined, accessKey=undefined, className=undefined, contentEditable=undefined, dir=undefined, draggable=undefined, hidden=undefined, lang=undefined, role=undefined, spellCheck=undefined, style=undefined, tabIndex=undefined, title=undefined, loading_state=undefined, **kwargs)
 |  
 |  A Div component.
 |  Div is a wrapper for the <div> HTML5 element.
 |  For detailed attribute info see:
 |  https://developer.mozilla.org/en-US/docs/Web/HTML/Element/div
 |  
 |  Keyword arguments:
 |  
 |  - children (a list of or a singular dash component, string or number; optional):
 |      The children of this component.
 |  
 |  - id (string; optional):
 |      The ID of this component, used to identify dash components in
 |      callbacks. The ID needs to be unique across all of the com